<img src="./resources/images/banner4.png" width="100%">

# Modelamiento y Evaluación
---

## **0. Integrantes del equipo de trabajo**
---

<table><thead>
  <tr>
    <th>#</th>
    <th>Integrante</th>
    <th>Documento de identidad</th>
  </tr></thead>
<tbody>
  <tr>
    <td>1</td>
    <td>Ivonne Cristina Ruiz Páez</td>
    <td>1014302058</td>  
  </tr>
  <tr>
    <td>2</td>
    <td>Diego Alejandro Feliciano Ramos</td>
    <td>1024586904</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Cristhian Enrique Córdoba Trillos</td>
    <td>1030649666</td>
  </tr>
</tbody>
</table>

## **1. Tipo de Modelamiento**
---

En el problema planteado queremos predecir el sentimiento que expresa cada post de redes sociales en el corpus, es decir, asignar una etiqueta dentro de un conjunto discreto (depresión, ansiedad, _normal_).
Por las siguientes razones el enfoque apropiado es clasificación supervisada:
* Naturaleza de la variable objetivo → es categórica, no continua; los enfoques de regresión quedarían descartados.
* Objetivo de negocio → necesitamos saber qué piensa la persona, no un número abstracto; un sistema de clasificación permite activar ‑por ejemplo‑ alertas de soporte si la probabilidad de “negativo” es alta.
* Disponibilidad de datos etiquetados → contamos con un historial de publicaciones que ya incluye la columna sentiment; eso habilita el entrenamiento supervisado.
* Escalabilidad y trazabilidad → clasificadores modernos (Logistic Regression, SVM, árboles de gradiente, etc.) pueden ree‑entrenarse con nueva data y ofrecen métricas claras (accuracy, F1, AUC).

Conclusión. Se selecciona la tarea de clasificación de texto. El resto del notebook se orienta a comparar varios algoritmos y seleccionar el que mejor macro‑F1 entregue sobre el conjunto de prueba.

## **2. Implementación del modelo**
---

En esta sección documentamos todo el flujo, desde la carga de datos hasta el entrenamiento del modelo final.
1.	Carga y pre‑procesamiento
    * Se descargan los datos (dataset.csv) y el sparse matrix de vectores TF‑IDF construido en la entrega anterior.
	* Se verifica consistencia de tamaños con X_vectores.shape y df_cargado.shape.
2.	Codificación de etiquetas
	* LabelEncoder transforma las clases de texto (sentiment) a enteros (y_numerico) para que los modelos de sklearn puedan operar.
3.	Catálogo de algoritmos a comparar
	* Siete clasificadores: Logistic Regression, Linear SVC, Multinomial NB, k‑NN, Random Forest, Gradient Boosting y XGBoost.
	* Cada uno se evalúa con una partición estratificada 80/20 y con validación cruzada (cross‑val score).
4.	Selección preliminar
	* Se calcula accuracy, precision_macro, recall_macro, f1_macro y roc_auc_ovr.
	* El algoritmo con mayor macro‑F1 pasa a la fase de ajuste fino.
5.	Ajuste de hiperparámetros (Grid Search)
	* Para XGBoost se exploran n_estimators, max_depth y learning_rate (ver celda de código).
	* Puntuación objetivo: f1_macro en validación 5‑fold.
6.	Entrenamiento final y persistencia
	* Con los mejores hiperparámetros hallados se re‑entrena el modelo en todo el training set.

### **2.1. Importar Modelo**
---
A continuación se realiza la descarga del conjunto de datos en formato CSV, el cual contiene una columna llamada sentiment. 
Este archivo se guardará localmente y luego se cargará con la librería pandas. El dataset servirá como entrada para entrenar o evaluar nuestro modelo.

In [1]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
# --- Importamos 7 modelos de la librería SKLearn ---
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
#
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import requests

In [3]:
# Cargamos los vectores de los documentos tokenizados de la entrega pasada.

url = "https://drive.google.com/uc?export=download&id=1XGw8FyvJM4Wu9_0lSoPDFiD_mXcUAXRm"
r = requests.get(url)
# Guarda el contenido descargado en un archivo local
with open("X_vectores.npy", "wb") as file:
    file.write(r.content)
# Ahora cargamos el archivo
X_vectores = np.load("X_vectores.npy")

In [6]:
# Cargamos el conjunto de datos inicial, donde se encuentra la columna de "sentiment"

url = "https://drive.google.com/uc?export=download&id=1MfG4qCfqAlj7JBWS0WmLQVOZZvY8wPRf"
r = requests.get(url)

# Guarda el contenido descargado en un archivo local
with open("dataset.csv", "wb") as code:
  code.write(r.content)
# Ahora cargamos el archivo
df_cargado = pd.read_csv("dataset.csv")

In [7]:
X_vectores.shape

(49710, 100)

In [8]:
df_cargado.shape

(49710, 3)

In [9]:
df_cargado.head()

,statement,status,clean_tokens
0,"trouble sleeping, confused mind, restless hear...",Anxiety,"['trouble', 'sleeping', 'confused', 'mind', 'r..."
1,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,"['wrong', 'back', 'dear', 'forward', 'doubt', ..."
2,I've shifted my focus to something else but I'...,Anxiety,"['shifted', 'focus', 'something', 'else', 'sti..."
3,"I'm restless and restless, it's been a month n...",Anxiety,"['restless', 'restless', 'month', 'boy', 'mean']"
4,"every break, you must be nervous, like somethi...",Anxiety,"['every', 'break', 'must', 'nervous', 'like', ..."


In [10]:
# Convertimos las etiquetas de texto en valores numéricos
label_encoder = LabelEncoder()
y_numerico = label_encoder.fit_transform(df_cargado['status'])

### **2.2. Entrenamiento del Modelo y Selección de Hiperparámetros**
---

In [11]:
# Definimos nuestro modelo con 'status' como la variable objetivo, y los vectores 'X_vectores'como nuestras variables explicativas.
# Definimos una decisión de 70/30 sobre nuestro universo de modelación, y creamos una semilla.
y = y_numerico
X_train, X_test, y_train, y_test = train_test_split(X_vectores, y, test_size=0.3, random_state=10)

In [12]:
# Definimos 7 modelos de clasificación para nuestro proyecto
modelos = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVC': SVC(probability=True, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

In [13]:
# Entrenamiento y evaluación de los modelos
resultados = {}

for nombre, modelo in modelos.items():
    print(f"\n🔵 Entrenando {nombre}...")
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    if hasattr(modelo, "predict_proba"):
        y_prob = modelo.predict_proba(X_test)
        try:
            auc = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')
        except ValueError:
            auc = np.nan
    else:
        auc = np.nan

    reporte = classification_report(y_test, y_pred, output_dict=True)
    resultados[nombre] = {
        'accuracy': reporte['accuracy'],
        'precision': reporte['macro avg']['precision'],
        'recall': reporte['macro avg']['recall'],
        'f1-score': reporte['macro avg']['f1-score'],
        'AUC': auc
    }
    print(classification_report(y_test, y_pred))


🔵 Entrenando Random Forest...
              precision    recall  f1-score   support

           0       0.78      0.59      0.67      1154
           1       0.77      0.40      0.53       835
           2       0.54      0.71      0.61      4537
           3       0.77      0.92      0.84      4157
           4       0.99      0.26      0.41       304
           5       0.97      0.19      0.32       822
           6       0.62      0.50      0.55      3104

    accuracy                           0.66     14913
   macro avg       0.78      0.51      0.56     14913
weighted avg       0.69      0.66      0.64     14913


🔵 Entrenando Decision Tree...
              precision    recall  f1-score   support

           0       0.50      0.48      0.49      1154
           1       0.37      0.41      0.39       835
           2       0.48      0.48      0.48      4537
           3       0.73      0.66      0.69      4157
           4       0.24      0.35      0.29       304
           5    

/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:42:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.77      0.66      0.71      1154
           1       0.76      0.53      0.62       835
           2       0.59      0.69      0.64      4537
           3       0.84      0.91      0.87      4157
           4       0.76      0.38      0.51       304
           5       0.64      0.37      0.47       822
           6       0.62      0.58      0.60      3104

    accuracy                           0.69     14913
   macro avg       0.71      0.59      0.63     14913
weighted avg       0.70      0.69      0.69     14913


🔵 Entrenando Logistic Regression...
              precision    recall  f1-score   support

           0       0.66      0.59      0.63      1154
           1       0.60      0.33      0.42       835
           2       0.53      0.71      0.61      4537
           3       0.74      0.77      0.75      4157
           4       0.35      0.10      0.15       304
           5       0.53      0.18      0.

In [14]:
# --- Mostrar resumen de resultados ---
df_resultados = pd.DataFrame(resultados).T
print("\n📈 Resumen de métricas:")
print(df_resultados.sort_values('f1-score', ascending=False))


📈 Resumen de métricas:
                     accuracy  precision    recall  f1-score       AUC
XGBoost              0.693355   0.712499  0.589103  0.632149  0.920049
Random Forest        0.657815   0.777454  0.508258  0.561345  0.896225
Gradient Boosting    0.662442   0.643303  0.523658  0.559509  0.905323
SVC                  0.679340   0.755921  0.503804  0.530403  0.915465
Logistic Regression  0.612687   0.573649  0.454718  0.482542  0.882614
K-Nearest Neighbors  0.528130   0.511449  0.442671  0.443048  0.788632
Decision Tree        0.503856   0.428724  0.444449  0.434641  0.677072


In [15]:
# Seleccionamos el modelo , como el mejor modelo de clasificación para nuestro conjunto de datos
modelo_seleccionado=XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=10)

In [16]:
# Definimos el espacio de los hiperparámetros a usar
param_grid = {
    'n_estimators': [50, 100, 150],  # Número de árboles
    'learning_rate': [0.01, 0.05, 0.1],  # Tasa de aprendizaje
    'max_depth': [3, 5, 7],  # Profundidad máxima de los árboles
    'min_child_weight': [1, 3, 5],  # Peso mínimo de cada nodo hijo
    'colsample_bytree': [0.7, 0.8, 1.0],  # Fracción de características
    'gamma': [0, 0.1, 0.2]  # Regularización
}

In [17]:
#Usamos grid search para la búsqueda de los hipeparámetros
grid_search = GridSearchCV(estimator=modelo_seleccionado, param_grid=param_grid,
                           scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:57:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:57:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:57:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:57:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constrain...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8, 1.0],
                         'gamma': [0, 0.1, 0.2],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

In [18]:
# Imprimimos los mejores hiperparámetros
print(f"Mejores hiperparámetros: {grid_search.best_params_}")

Mejores hiperparámetros: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 150}


## **3. Evaluación o Aplicación del modelo**
---

Si entrenó un modelo, recuerde que debe reportar el desempeño del mismo sobre un conjunto de datos no visto (test). Considere que dispone de las siguientes métricas:

- **Clasificación**: accuracy, precision, recall, f1-score, AUC.
- **Regresión**: $r^2$, error cuadrático medio, error absoluto medio.
- **Agrupamiento**: coeficiente de silueta, índice de Davies-Bouldin.
- **Tópicos**: perplexity, score de coherencia.

In [19]:
# Evaluamos en conjunto de prueba
mejor_modelo = grid_search.best_estimator_
y_pred = mejor_modelo.predict(X_test)
y_prob = mejor_modelo.predict_proba(X_test)

In [20]:
# Reporte de clasificación
print("\n📊 Clasificación en conjunto de prueba:")
print(classification_report(y_test, y_pred))


📊 Clasificación en conjunto de prueba:
              precision    recall  f1-score   support

           0       0.77      0.65      0.70      1154
           1       0.77      0.52      0.62       835
           2       0.59      0.70      0.64      4537
           3       0.83      0.92      0.87      4157
           4       0.74      0.35      0.47       304
           5       0.66      0.34      0.45       822
           6       0.64      0.58      0.61      3104

    accuracy                           0.69     14913
   macro avg       0.71      0.58      0.62     14913
weighted avg       0.70      0.69      0.69     14913



In [21]:
# AUC
auc = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')
print(f"AUC: {auc}")

AUC: 0.9208293027256556


## **Créditos**

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*